In [1]:
from qiskit import *
IBMQ.load_account()
simulator = BasicAer.get_backend('qasm_simulator')

/usr/local/lib/python3.7/dist-packages/qiskit/__init__.py:65: RuntimeWarning: Could not import the Aer provider from the qiskit-aer package. Install qiskit-aer or check your installation.
  RuntimeWarning)
/usr/local/lib/python3.7/dist-packages/qiskit/providers/ibmq/ibmqfactory.py:192: UserWarning: Timestamps in IBMQ backend properties, jobs, and job results are all now in local time instead of UTC.
  warnings.warn('Timestamps in IBMQ backend properties, jobs, and job results '


In [2]:
# Task 2
from math import *
circuit = QuantumCircuit(5, 4)
theta = pi / 5
circuit.x(4)
circuit.h(0)
circuit.h(1)
circuit.h(2)
circuit.h(3)
circuit.barrier()
# to make things shorter, U^2(theta) = U(2 * theta)
circuit.cu1(theta, 0, 4)
circuit.cu1(2 * theta, 1, 4)
circuit.cu1(4 * theta, 2, 4)
circuit.cu1(8 * theta, 3, 4)
circuit.barrier()
# inverse qft
w = -pi
circuit.swap(0, 3)
circuit.swap(1, 2)
circuit.h(0)
circuit.cu1(w / 2, 0, 1)
circuit.cu1(w / 4, 0, 2)
circuit.cu1(w / 8, 0, 3)
circuit.barrier()
circuit.h(1)
circuit.cu1(w / 2, 1, 2)
circuit.cu1(w / 4, 1, 3)
circuit.barrier()
circuit.h(2)
circuit.cu1(w / 2, 2, 3)
circuit.h(3)
circuit.barrier()
circuit.measure([0, 1, 2, 3], [0, 1, 2, 3])
circuit.draw()

┌───┐ ░                                 ░    ┌───┐                        »
q_0: ┤ H ├─░──■──────────────────────────────░──X─┤ H ├─■───────■───────■──────»
     ├───┤ ░  │                              ░  │ └───┘ │-pi/2  │       │      »
q_1: ┤ H ├─░──┼──────■───────────────────────░──┼───X───■───────┼───────┼──────»
     ├───┤ ░  │      │                       ░  │   │           │-pi/4  │      »
q_2: ┤ H ├─░──┼──────┼───────■───────────────░──┼───X───────────■───────┼──────»
     ├───┤ ░  │      │       │               ░  │                       │-pi/8 »
q_3: ┤ H ├─░──┼──────┼───────┼───────■───────░──X───────────────────────■──────»
     ├───┤ ░  │pi/5  │2pi/5  │4pi/5  │8pi/5  ░                                 »
q_4: ┤ X ├─░──■──────■───────■───────■───────░─────────────────────────────────»
     └───┘ ░                                 ░                                 »
c: 4/══════════════════════════════════════════════════════════════════════════»
                                                                               »
«      ░                       ░                    ░ ┌─┐         
«q_0: ─░───────────────────────░────────────────────░─┤M├─────────
«      ░ ┌───┐                 ░                    ░ └╥┘┌─┐      
«q_1: ─░─┤ H ├─■───────■───────░────────────────────░──╫─┤M├──────
«      ░ └───┘ │-pi/2  │       ░ ┌───┐              ░  ║ └╥┘┌─┐   
«q_2: ─░───────■───────┼───────░─┤ H ├─■────────────░──╫──╫─┤M├───
«      ░               │-pi/4  ░ └───┘ │-pi/2 ┌───┐ ░  ║  ║ └╥┘┌─┐
«q_3: ─░───────────────■───────░───────■──────┤ H ├─░──╫──╫──╫─┤M├
«      ░                       ░              └───┘ ░  ║  ║  ║ └╥┘
«q_4: ─░───────────────────────░────────────────────░──╫──╫──╫──╫─
«      ░                       ░                    ░  ║  ║  ║  ║ 
«c: 4/═════════════════════════════════════════════════╩══╩══╩══╩═
«                                                      0  1  2  3

In [3]:
job = execute(circuit, simulator, shots=65536)
result = job.result()
counts = result.get_counts(circuit)
for x, y in sorted(counts.items()):
    print(x, y)
ans = int(max([(y, x) for x, y in counts.items()])[1], 2)
print("theta = {}, correct is {}".format(ans / (2 ** 4) * 2 * pi, theta))

0000 2382
0001 16597
0010 37737
0011 3221
0100 1120
0101 611
0110 414
0111 310
1000 278
1001 220
1010 213
1011 276
1100 298
1101 349
1110 537
1111 973
theta = 0.7853981633974483, correct is 0.6283185307179586


In [22]:
# Task 1.2
simulator = BasicAer.get_backend('unitary_simulator')
circuit = QuantumCircuit(3, 3)
x = 3
n = 3
# prepare state |x>
for i in range(n):
    if (x >> i) & 1:
        circuit.x(i)
# after QFT amplitude for k should be 1/2sqrt(2) exp(2 * pi * i * k * x / 8)
circuit.barrier()
# qft
w = pi
circuit.swap(0, 2)
circuit.h(0)
circuit.cu1(w / 2, 0, 1)
circuit.cu1(w / 4, 0, 2)
circuit.barrier()
circuit.h(1)
circuit.cu1(w / 2, 1, 2)
circuit.barrier()
circuit.h(2)
circuit.draw()

┌───┐ ░    ┌───┐               ░              ░      
q_0: ┤ X ├─░──X─┤ H ├─■──────■──────░──────────────░──────
     ├───┤ ░  │ └───┘ │pi/2  │      ░ ┌───┐        ░      
q_1: ┤ X ├─░──┼───────■──────┼──────░─┤ H ├─■──────░──────
     └───┘ ░  │              │pi/4  ░ └───┘ │pi/2  ░ ┌───┐
q_2: ──────░──X──────────────■──────░───────■──────░─┤ H ├
           ░                        ░              ░ └───┘
c: 3/═════════════════════════════════════════════════════

In [23]:
import cmath
job = execute(circuit, simulator)
result = job.result()
res = result.get_unitary(circuit, decimals=3)
res = [x[0] for x in res]
for k in range(2 ** n):
    print("phase expected: {}, phase computed: {}".format((2 * pi * k * x / (2 ** n)) % (2 * pi), cmath.phase(res[k]) % (2 * pi)))

phase expected: 0.0, phase computed: 6.283185307179586
phase expected: 2.356194490192345, phase computed: 2.3561944901923444
phase expected: 4.71238898038469, phase computed: 4.71238898038469
phase expected: 0.7853981633974483, phase computed: 0.7853981633974478
phase expected: 3.141592653589793, phase computed: 3.141592653589793
phase expected: 5.497787143782137, phase computed: 5.497787143782137
phase expected: 1.5707963267948966, phase computed: 1.5707963267948963
phase expected: 3.9269908169872423, phase computed: 3.926990816987241
